<a href="https://colab.research.google.com/github/seawavve/EMOmap/blob/main/Comparing_two_EMOmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
! pip install chart-studio

In [54]:
from chart_studio import plotly as py
import plotly.graph_objects as go
import pandas as pd
import numpy

def make_fig(x_list,y_list,y_list2):
  fig1=go.Scatter( x=x_list,y=y_list )
  fig2=go.Scatter( x=x_list,y=y_list2 )
  data=[fig1,fig2]
  fig = go.Figure(data)
  fig.show()

def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    import numpy as np
    from math import factorial

    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))

    except ValueError:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

def get_y_list():
  data=pd.read_csv('./Pinocchio_API.csv')
  mul = data['magnitude'].mul(data['score'])
  data['mul']=mul*100
  #display(data)
  return data['mul'].tolist()

def get_y_list2():
  data=pd.read_csv('./Junglebook_API.csv')
  mul = data['magnitude'].mul(data['score'])
  data['mul']=mul*100
  #display(data)
  return data['mul'].tolist()


def main():
  y_list=numpy.array(get_y_list())
  y_list=savitzky_golay(y_list, window_size=101, order=4)

  y_list2=numpy.array(get_y_list2())
  for _ in range(len(y_list2)-len(y_list)):
    y_list2.append(0)
  y_list2=savitzky_golay(y_list2, window_size=101, order=4)
  
  x_list=list(range(len(y_list)))
  print(type(x_list),type(y_list))
  make_fig(x_list,y_list,y_list2)
  return 0

if __name__ == "__main__":
	main()

<class 'list'> <class 'numpy.ndarray'>
